In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = '/content/drive/MyDrive/book-crossings/BX-Books.csv'
ratings_filename = '/content/drive/MyDrive/book-crossings/BX-Book-Ratings.csv'

--2024-11-27 02:50:21--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   118MB/s    in 0.2s    

2024-11-27 02:50:21 (118 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [73]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import zipfile

# Step 1: Load and Extract Dataset
zip_path = '/content/drive/MyDrive/book-crossings.zip'  # Replace with your ZIP file path
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall('/content/')

# Load datasets
books = pd.read_csv('/content/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
ratings = pd.read_csv('/content/BX-Book-Ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)

# Step 2: Data Cleaning
# Filter users with fewer than 200 ratings
user_ratings_count = ratings['User-ID'].value_counts()
active_users = user_ratings_count[user_ratings_count >= 200].index
ratings = ratings[ratings['User-ID'].isin(active_users)]

# Filter books with fewer than 100 ratings
book_ratings_count = ratings['ISBN'].value_counts()
popular_books = book_ratings_count[book_ratings_count >= 100].index
ratings = ratings[ratings['ISBN'].isin(popular_books)]

# Step 3: Create Book-User Matrix
book_user_matrix = ratings.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)

# Step 4: Fit the KNN Model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(book_user_matrix)

# Step 5: Define get_recommends Function
def get_recommends(book_title):
    # Ensure books remains a DataFrame
    global books
    if not isinstance(books, pd.DataFrame):
        books = pd.read_csv('/content/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)

    # Map book title to ISBN
    book_mapping = books.set_index('Book-Title')['ISBN'].to_dict()
    isbn_mapping = books.set_index('ISBN')['Book-Title'].to_dict()

    # Check if the book exists in the mapping
    if book_title not in book_mapping:
        return [book_title, "Book not found in dataset."]

    # Proceed if the book is found
    isbn = book_mapping[book_title]
    if isbn not in book_user_matrix.index:
        return [book_title, "Book not found in matrix."]

    book_index = book_user_matrix.index.tolist().index(isbn)
    distances, indices = knn.kneighbors(
        book_user_matrix.iloc[book_index, :].values.reshape(1, -1),
        n_neighbors=6
    )

    similar_books = []
    for i in range(1, len(indices.flatten())):
        similar_isbn = book_user_matrix.index[indices.flatten()[i]]
        similar_books.append([isbn_mapping[similar_isbn], distances.flatten()[i]])

    return [book_title, similar_books]

In [78]:
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    expected_books = ["The Lovely Bones: A Novel", "The Pilot's Wife : A Novel", "The Joy Luck Club", "The Notebook", "Bel Canto: A Novel"]
    expected_distances = [0.7234864549790632, 0.8192678336213406, 0.8198604785739199, 0.8236682900571164, 0.8247874880681115]
    for i, book in enumerate(expected_books):
        if i >= len(recommends[1]) or recommends[1][i][0] != book:
            test_pass = False
    for i, dist in enumerate(expected_distances):
        if i >= len(recommends[1]) or abs(recommends[1][i][1] - dist) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")
recommendations = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print("Generated Recommendations:", recommendations)

test_book_recommendation()

Generated Recommendations: ["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864549790632], ["The Pilot's Wife : A Novel", 0.8192678336213406], ['The Joy Luck Club', 0.8198604785739199], ['The Notebook', 0.8236682900571164], ['Bel Canto: A Novel', 0.8247874880681115]]]
You passed the challenge! 🎉🎉🎉
